In [ ]:
from google.colab.auth import authenticate_user
authenticate_user()

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

In [ ]:
!gsutil cp gs://ekaba-assets/datasets/QA/BioASQ/BioASQ-train-factoid-4b.json .
!gsutil cp gs://ekaba-assets/datasets/QA/BioASQ/BioASQ-train-factoid-5b.json .
!gsutil cp gs://ekaba-assets/datasets/QA/BioASQ/BioASQ-train-factoid-6b.json .
!gsutil cp gs://ekaba-assets/datasets/QA/BioASQ/training7b.json .

In [ ]:
df1 = pd.read_json('BioASQ-train-factoid-4b.json')
df2 = pd.read_json('BioASQ-train-factoid-5b.json')
df3 = pd.read_json('BioASQ-train-factoid-6b.json')
df4 = pd.read_json('training7b.json')

In [ ]:
l1 = len(df1['data'].iloc[0]['paragraphs'])
l2 = len(df2['data'].iloc[0]['paragraphs'])
l3 = len(df3['data'].iloc[0]['paragraphs'])
l4 = len(df4['data'].iloc[0]['paragraphs'])

In [ ]:
def get_df(l,df) :

    question = []
    q_id = []
    ans = []

    for i in tqdm(range(0,l)):

        para = df['data'].iloc[0]['paragraphs'][i]
        qas = para['qas'][0]

        question.append(qas['question'])
        q_id.append(qas['id'])
        ans.append(para['context'])
        
    
    
    #Dataframe
    df_qa = pd.DataFrame()
    
    
    #dataframe
    df_qa['id'] = q_id
    df_qa['question'] = question
    df_qa['answer'] = ans

    return df_qa



In [ ]:
df_qa1 = get_df(l1,df1)
df_qa2 = get_df(l2,df2)
df_qa3 = get_df(l3,df3)
df_qa4 = get_df(l4,df4)

In [ ]:
df_qa = pd.concat([df_qa1,df_qa2,df_qa3,df_qa4],sort=False)

In [ ]:
df_qa = df_qa.drop(['id'],axis=1)

In [ ]:
df_qa['answer'].nunique()

In [ ]:
df_qa_un = df_qa.drop_duplicates(keep='first')

In [ ]:
df_qa_un.head()

In [ ]:
def text_preprocessing(text):
    text= text.lower()
    text= re.sub(r'[^a-z]',' ',text)
    #text= " ".join([s for s in text.split() if len(s)>2])
    #text= " ".join([x for x in text.split() if x not in stopwords.words('english')])
    #text= " ".join([inflection.singularize(x) for x in text.split()])
    text= text.strip()
    return(text)

In [ ]:
tqdm.pandas()

In [ ]:
df_qa_un['question'] = df_qa_un['question'].progress_apply(lambda x: text_preprocessing(x))
df_qa_un['answer'] =   df_qa_un['answer'].progress_apply(lambda x: text_preprocessing(x))

In [ ]:
df_qa_un.shape

In [ ]:
unique_ques = list(df_qa_un['question'].unique())
unique_ans =  list(df_qa_un['answer'].unique())

In [ ]:
dict_ques = {}

for i,qs in enumerate(unique_ques):
    dict_ques[qs] = i    

In [ ]:
dict_ans = {}

for i,ans in enumerate(unique_ans):
    dict_ans[ans] = i    

In [ ]:
qid = []
pid = []


for i in tqdm(range(0,len(df_qa_un))):
    
    ques = df_qa_un['question'].iloc[i]
    ans = df_qa_un['answer'].iloc[i]
    
    qid.append(dict_ques[ques])
    pid.append(dict_ans[ans])

In [ ]:
df_qa_un['qid'] = qid
df_qa_un['pid'] = pid

In [ ]:
df_qa_un.head()

In [ ]:
df_qa_un[df_qa_un['answer'].str.contains('fraumeni')]

In [ ]:
df_qa_un[df_qa_un['answer'].str.contains('status')]

In [ ]:
#df_qa1['qid'].value_counts()

In [ ]:
#df_qa1['pid'].value_counts()

We need to have more answers per question to have a more robust MRR metric. We will add more data synthetically to enhance the quality of the metric